# Interactive Weights & Biases Tutorial

This notebook will guide you through setting up and using Weights & Biases (W&B) for experiment tracking. Each cell is executable and includes real-time feedback.

## Table of Contents
1. [Setup and Installation](#setup)
2. [First Experiment](#first-experiment)
3. [Advanced Tracking](#advanced-tracking)
4. [Hyperparameter Optimization](#hyperparameter-optimization)

## 1. Setup and Installation <a name="setup"></a>

In [ ]:
def check_wandb_installation():
    try:
        import wandb
        print(f"✅ W&B is installed (version {wandb.__version__})")
        return True
    except ImportError:
        print("❌ W&B is not installed. Installing now...")
        !pip install wandb -q
        return False

check_wandb_installation()

### Login to W&B
Run this cell to log in to your W&B account. If you don't have one, it will guide you through creating one.

In [ ]:
import wandb
wandb.login()

## 2. First Experiment <a name="first-experiment"></a>

Let's create a simple experiment to understand how W&B tracking works:

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Create synthetic data
X = torch.randn(100, 1)
y = 2 * X + 1 + 0.1 * torch.randn(100, 1)

# Define a simple model
model = nn.Linear(1, 1)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Initialize W&B
wandb.init(
    project="interactive-tutorial",
    config={
        "learning_rate": 0.01,
        "epochs": 100,
        "batch_size": 32
    }
)

# Training loop
for epoch in range(wandb.config.epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    
    # Log metrics to W&B
    wandb.log({
        "loss": loss.item(),
        "weight": model.weight.item(),
        "bias": model.bias.item()
    })

wandb.finish()

## 3. Advanced Tracking <a name="advanced-tracking"></a>

Now let's explore more advanced W&B features:

In [ ]:
import matplotlib.pyplot as plt

# Initialize a new run
wandb.init(
    project="interactive-tutorial",
    name="advanced-tracking",
    config={
        "architecture": "LinearRegression",
        "dataset": "synthetic",
        "learning_rate": 0.01
    }
)

# Create and log a custom plot
plt.figure(figsize=(10, 6))
plt.scatter(X.numpy(), y.numpy(), label='True Data')
plt.plot(X.numpy(), model(X).detach().numpy(), 'r', label='Prediction')
plt.legend()
plt.title('Regression Results')

# Log the plot to W&B
wandb.log({"regression_plot": wandb.Image(plt)})

# Log model architecture
wandb.watch(model)

# Create a custom table
data = [[x.item(), y_.item(), pred.item()] 
        for x, y_, pred in zip(X[:5], y[:5], model(X[:5]))]
table = wandb.Table(data=data, columns=["input", "true", "predicted"])
wandb.log({"predictions": table})

wandb.finish()

## 4. Hyperparameter Optimization <a name="hyperparameter-optimization"></a>

Let's use W&B Sweeps for hyperparameter optimization:

In [ ]:
sweep_configuration = {
    'method': 'random',
    'metric': {'name': 'loss', 'goal': 'minimize'},
    'parameters': {
        'learning_rate': {'values': [0.001, 0.01, 0.1]},
        'batch_size': {'values': [16, 32, 64]},
        'epochs': {'value': 50}
    }
}

sweep_id = wandb.sweep(sweep_configuration, project="interactive-tutorial")

def train_model():
    # Initialize a new wandb run
    wandb.init()
    
    # Create model with current hyperparameters
    model = nn.Linear(1, 1)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=wandb.config.learning_rate)
    
    # Training loop
    for epoch in range(wandb.config.epochs):
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        
        wandb.log({"loss": loss.item()})

# Run the sweep
wandb.agent(sweep_id, function=train_model, count=5)

## Next Steps

Now that you've completed this interactive tutorial, you can:
1. Visit your W&B dashboard to see your experiments
2. Explore the sweep results and visualizations
3. Try integrating W&B into your own projects

For more advanced features, check out:
- [W&B Reports](https://docs.wandb.ai/guides/reports)
- [Artifact Management](https://docs.wandb.ai/guides/artifacts)
- [Team Collaboration](https://docs.wandb.ai/guides/teams)